# MLP Clustering Significance Stability over Multiple Trainings - Different #Clusters

### Before Running on Perceptron

Go to `src/train_nn.py` to line 46, and change the path `./models/` to `/scratch/<your username>/models`

Do not forget to undo this change!

DO NOT COMMIT THIS CHANGE TO GITHUB!!!

In [1]:
N_CLUSTERS = 4

In [2]:
# Parameters
N_CLUSTERS = 2


In [3]:
%load_ext autoreload
%autoreload 1

In [4]:
import sys
sys.path.append('..')

import random
import subprocess
from multiprocessing import Pool

import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from tqdm import tqdm, trange
from IPython import display

from src.visualization import draw_mlp_clustering_report, run_double_spectral_cluster, run_spectral_cluster
from src.utils import get_weights_paths, build_clustering_results, cohen_d_stats
from src.experiment_tagging import get_model_path, MODEL_TAG_LOOKUP
# from src.spectral_cluster_model import SHUFFLE_METHODS
SHUFFLE_METHODS = ['layer', 'layer_nonzero']

In [5]:
N_TRAINED_MODELS = 10

# we ar doing 320 experiments (=32 models x 10 times),
# and let's have expectation of false-positive <= 1
# so we can set n_shuffles to 320, and then p-values < 1/401
# if we make it too large, let see, 1000 shuffles...
# ... we might reduce the statistical power (given H1, the probability to reject it incorrectly)!
N_SHUFFLES = 320

N_GPUS = 4

TAGS_COMMANDS = {
    'MNIST': 'python -m src.train_nn with mlp_config dataset_name=mnist',
    'CIFAR10': 'python -m src.train_nn with mlp_config dataset_name=cifar10 pruning_epochs=40',
#    'LINE': 'python -m src.train_nn with mlp_config dataset_name=line',
    'FASHION': 'python -m src.train_nn with mlp_config dataset_name=fashion',
    'MNIST+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist with_dropout=True',
    'CIFAR10+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=cifar10 epochs=100 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'LINE+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line with_dropout=True',
    'FASHION+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=fashion with_dropout=True',
#    'LINE-MNIST': 'python -m src.train_nn with mlp_config dataset_name=line-mnist',
#    'LINE-CIFAR10': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10 epochs=30 pruning_epochs=40',
#    'MNIST-CIFAR10': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10 epochs=30 pruning_epochs=40',
#    'LINE-MNIST-SEPARATED': 'python -m src.train_nn with mlp_config dataset_name=line-mnist-separated',
#    'LINE-CIFAR10-SEPARATED': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10-separated epochs=30 pruning_epochs=40',
#    'MNIST-CIFAR10-SEPARATED': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10-separated epochs=30 pruning_epochs=40',
#    'LINE-MNIST+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-mnist with_dropout=True',
#    'LINE-CIFAR10+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10 epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'MNIST-CIFAR10+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10 epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'LINE-MNIST-SEPARATED+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-mnist-separated with_dropout=True',
#    'LINE-CIFAR10-SEPARATED+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10-separated epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'MNIST-CIFAR10-SEPARATED+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10-separated epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'RANDOM': 'python -m src.train_nn with mlp_config dataset_name=random',
#    'RANDOM+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=random with_dropout=True',
#    'MNIST-x1.5-EPOCHS': 'python -m src.train_nn with mlp_config dataset_name=mnist epochs=30',
#    'MNIST-x1.5-EPOCHS+DROPOUT':'python -m src.train_nn with mlp_config dataset_name=mnist epochs=30 with_dropout=True',
#    'MNIST-x2-EPOCHS':'python -m src.train_nn with mlp_config dataset_name=mnist epochs=40',
#    'MNIST-x2-EPOCHS+DROPOUT':'python -m src.train_nn with mlp_config dataset_name=mnist epochs=40 with_dropout=True',
#    'MNIST-x10-EPOCHS': 'python -m src.train_nn with mlp_config dataset_name=mnist epochs=200',
#    'MNIST-x10-EPOCHS+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist epochs=200 with_dropout=True',
#    'RANDOM-x50-EPOCHS': 'python -m src.train_nn with mlp_config dataset_name=random epochs=1000',
#    'RANDOM-x50-EPOCHS+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=random epochs=1000 with_dropout=True',
#    'RANDOM-OVERFITTING': 'python -m src.train_nn with mlp_config dataset_name=random epochs=100 pruning_epochs=100 shuffle=False n_train=3000',
#    'RANDOM-OVERFITTING+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=random epochs=100 pruning_epochs=100 shuffle=False n_train=3000 with_dropout=True'
}
DATASETS_TAGS = [command.split()[5][13:] for command in TAGS_COMMANDS.values()]

In [6]:
if False:
    def train_model_multiply(command, n_models_per_command, gpu_id=None):    
        for _ in range(n_models_per_command):
            actual_command = f'cd .. && {command}'#' > /dev/null 2>&1'
            print(actual_command)
            subprocess.run(actual_command, shell=True,
                           env={'CUDA_VISIBLE_DEVICES': str(gpu_id)})


    def train_models(commands, n_models_per_command, gpu_id=None):
        for command in commands:
            train_model_multiply(command, n_models_per_command, gpu_id)

            commands = list(TAGS_COMMANDS.values())
    random.shuffle(commands)

    assert len(commands) % N_GPUS == 0
    n_commands_per_gpu = len(commands) // N_GPUS

    with Pool(N_GPUS) as p:

        p.starmap(train_models,
                 [(commands[gpu_id*n_commands_per_gpu : (gpu_id+1)*n_commands_per_gpu], N_TRAINED_MODELS, gpu_id)
                 for gpu_id in range(N_GPUS)])

In [7]:
results = {}

for shuffle_method in SHUFFLE_METHODS:
    results[shuffle_method] = {}
    
    for (model_tag, _), dataset_tag in zip(TAGS_COMMANDS.items(), tqdm(DATASETS_TAGS)):
        model_paths = get_model_path(model_tag, filter_='all', model_base_path='/scratch/shlomi/models/')


        results[shuffle_method][model_tag] = [run_spectral_cluster(path / f'{dataset_tag}-mlp-pruned-weights.pckl',
                                               n_samples=N_SHUFFLES,
                                               shuffle_method=shuffle_method,
                                               n_clusters=N_CLUSTERS)
            for path in tqdm(model_paths[-N_TRAINED_MODELS:])]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:25<03:52, 25.88s/it]

 20%|██        | 2/10 [00:51<03:25, 25.66s/it]

 30%|███       | 3/10 [01:16<02:58, 25.54s/it]

...

 70%|███████   | 7/10 [02:58<01:16, 25.65s/it]

 80%|████████  | 8/10 [03:23<00:50, 25.34s/it]

 90%|█████████ | 9/10 [03:48<00:25, 25.44s/it]

100%|██████████| 10/10 [04:14<00:00, 25.54s/it]

100%|██████████| 10/10 [04:14<00:00, 25.47s/it]

In [8]:
run_spectral_cluster?

In [9]:
def build_all_models_per_shuffle_method_df(single_shuffle_method_results):
    model_dfs = []

    for model_tag, model_results in single_shuffle_method_results.items():
        _, metrics  = zip(*model_results)
        model_dfs.append(pd.DataFrame(metrics).assign(model=model_tag))

    df = pd.concat(model_dfs, ignore_index=True)

    df['is_sig'] = np.isclose(df['percentile'], 1 / (N_SHUFFLES + 1))
    
    return df

layer_df = build_all_models_per_shuffle_method_df(results['layer'])
layer_nonzero_df = build_all_models_per_shuffle_method_df(results['layer_nonzero'])

In [10]:
layer_df.head()

,ave_in_out,mean,n_samples,ncut,percentile,stdev,test_acc,test_loss,train_acc,train_loss,z_score,model,is_sig
0,2.530629,0.329872,320,0.329965,0.563863,0.002359,0.9835,0.095332,1.0,0.000048,0.039226,MNIST,False
1,2.528825,0.329599,320,0.330161,0.563863,0.002051,0.9847,0.090697,1.0,0.000056,0.273888,MNIST,False
2,2.494644,0.328610,320,0.333930,0.968847,0.002429,0.9830,0.099833,1.0,0.000059,2.190334,MNIST,False
3,2.519351,0.329903,320,0.331197,0.626168,0.002327,0.9828,0.099546,1.0,0.000062,0.556271,MNIST,False
4,2.462010,0.332344,320,0.337609,1.000000,0.002243,0.9845,0.097756,1.0,0.000081,2.347679,MNIST,False


In [11]:
layer_nonzero_df.head()

,ave_in_out,mean,n_samples,ncut,percentile,stdev,test_acc,test_loss,train_acc,train_loss,z_score,model,is_sig
0,2.530629,0.329494,320,0.329965,0.595016,0.001083,0.9835,0.095332,1.0,0.000048,0.434404,MNIST,False
1,2.528825,0.328940,320,0.330161,0.875389,0.000903,0.9847,0.090697,1.0,0.000056,1.351846,MNIST,False
2,2.494644,0.332132,320,0.333930,0.968847,0.001124,0.9830,0.099833,1.0,0.000059,1.599889,MNIST,False
3,2.519351,0.329517,320,0.331197,1.000000,0.000927,0.9828,0.099546,1.0,0.000062,1.812551,MNIST,False
4,2.462010,0.335694,320,0.337609,0.875389,0.001318,0.9845,0.097756,1.0,0.000081,1.452518,MNIST,False


In [12]:
MODEL_DATA_COLUMNS = ['model',
                      'train_acc', 'train_loss', 'test_acc', 'test_loss',
                      'ncut', 'ave_in_out',
                      'n_samples']

assert layer_df[MODEL_DATA_COLUMNS].equals(layer_nonzero_df[MODEL_DATA_COLUMNS])

SHUFFLING_DATA_COLUMNS = ['mean', 'percentile', 'stdev', 'z_score', 'is_sig']

two_shuffling_method_df = pd.merge(layer_df[SHUFFLING_DATA_COLUMNS],
                                     layer_nonzero_df[SHUFFLING_DATA_COLUMNS],
                                     left_index=True, right_index=True,
                                     suffixes=('_layer', '_layer_nonzero'))

assert layer_df[MODEL_DATA_COLUMNS].equals(layer_nonzero_df[MODEL_DATA_COLUMNS])

df = pd.merge(layer_df[MODEL_DATA_COLUMNS], two_shuffling_method_df,
              left_index=True, right_index=True,)

df['cohen_d'] = df.apply(lambda r: cohen_d_stats(r['mean_layer'], r['stdev_layer'], r['n_samples'],
                                 r['mean_layer_nonzero'], r['stdev_layer_nonzero'], r['n_samples']),
                                axis=1)

df

,model,train_acc,train_loss,test_acc,test_loss,ncut,ave_in_out,n_samples,mean_layer,percentile_layer,stdev_layer,z_score_layer,is_sig_layer,mean_layer_nonzero,percentile_layer_nonzero,stdev_layer_nonzero,z_score_layer_nonzero,is_sig_layer_nonzero,cohen_d
0,MNIST,1.000000,0.000048,0.9835,0.095332,0.329965,2.530629,320,0.329872,0.563863,0.002359,0.039226,False,0.329494,0.595016,0.001083,0.434404,False,0.483902
1,MNIST,1.000000,0.000056,0.9847,0.090697,0.330161,2.528825,320,0.329599,0.563863,0.002051,0.273888,False,0.328940,0.875389,0.000903,1.351846,False,0.518073
2,MNIST,1.000000,0.000059,0.9830,0.099833,0.333930,2.494644,320,0.328610,0.968847,0.002429,2.190334,False,0.332132,0.968847,0.001124,1.599889,False,0.474365
3,MNIST,1.000000,0.000062,0.9828,0.099546,0.331197,2.519351,320,0.329903,0.626168,0.002327,0.556271,False,0.329517,1.000000,0.000927,1.812551,False,0.513064
4,MNIST,1.000000,0.000081,0.9845,0.097756,0.337609,2.462010,320,0.332344,1.000000,0.002243,2.347679,False,0.335694,0.875389,0.001318,1.452518,False,0.450070
5,MNIST,1.000000,0.000048,0.9835,0.095626,0.331404,2.517462,320,0.331395,0.408100,0.002754,0.003563,False,0.328806,1.000000,0.000848,3.063931,False,0.531824
6,MNIST,1.000000,0.000056,0.9838,0.087238,0.335151,2.483726,320,0.330888,0.968847,0.002116,2.014740,False,0.333080,1.000000,0.001018,2.033987,False,0.495887
7,MNIST,1.000000,0.000061,0.9838,0.100444,0.332732,2.505426,320,0.332105,0.626168,0.001924,0.325583,False,0.333334,0.221184,0.000755,-0.797120,False,0.554175
8,MNIST,1.000000,0.000047,0.9828,0.093142,0.331260,2.518777,320,0.330194,0.657321,0.001992,0.535228,False,0.329284,0.937695,0.001082,1.826329,False,0.485221
9,MNIST,1.000000,0.000049,0.9833,0.102283,0.337271,2.464977,320,0.330002,0.968847,0.002464,2.949653,False,0.335770,0.844237,0.001393,1.077225,False,0.436860


In [13]:
# df = pd.read_csv('../results/mlp-clustering-stability-two-shuffling-methods-all-samples.csv' index_col=0)

In [14]:
df_grpby_model = df.groupby('model')

TWO_SHUFFLE_STATS_COLUMNS = [f'{stat}_{shuffle_method}'
                             for stat in ['mean', 'stdev', 'z_score', 'percentile', 'is_sig']
                             for shuffle_method in SHUFFLE_METHODS]

statistics_df = pd.concat([(df_grpby_model[['ncut'] + TWO_SHUFFLE_STATS_COLUMNS]
                             .agg(['mean', 'std'])),
                            df_grpby_model.size().rename('n_models'),
                            (df_grpby_model[['train_acc', 'test_acc']]
                             .agg('mean'))],
                           axis=1)

In [15]:
statistics_df

,"(ncut, mean)","(ncut, std)","(mean_layer, mean)","(mean_layer, std)","(mean_layer_nonzero, mean)","(mean_layer_nonzero, std)","(stdev_layer, mean)","(stdev_layer, std)","(stdev_layer_nonzero, mean)","(stdev_layer_nonzero, std)",...,"(percentile_layer, std)","(percentile_layer_nonzero, mean)","(percentile_layer_nonzero, std)","(is_sig_layer, mean)","(is_sig_layer, std)","(is_sig_layer_nonzero, mean)","(is_sig_layer_nonzero, std)",n_models,train_acc,test_acc
model,,,,,,,,,,,,,,,,,,,,,
CIFAR10,0.306362,0.009507,0.312830,0.006120,0.296695,0.013320,0.001239,0.000184,0.000581,0.000188,...,0.078811,1.000000,0.000000,0.9,0.316228,False,0.0,10,0.650088,0.41523
CIFAR10+DROPOUT,0.289859,0.014990,0.308491,0.008694,0.257023,0.024589,0.000955,0.000167,0.001108,0.000193,...,0.019703,1.000000,0.000000,0.9,0.316228,False,0.0,10,0.427050,0.42189
FASHION,0.319421,0.003331,0.312592,0.001731,0.317683,0.002728,0.002186,0.000274,0.000926,0.000157,...,0.013135,0.909657,0.174659,0.0,0.000000,False,0.0,10,0.982630,0.89344
FASHION+DROPOUT,0.311667,0.003239,0.312478,0.001810,0.311663,0.002760,0.001896,0.000245,0.001072,0.000263,...,0.362542,0.476636,0.356649,0.0,0.000000,False,0.0,10,0.863268,0.86921
MNIST,0.333068,0.002811,0.330491,0.001174,0.331605,0.002762,0.002266,0.000253,0.001045,0.000201,...,0.218439,0.831776,0.246699,0.0,0.000000,False,0.0,10,1.000000,0.98357
MNIST+DROPOUT,0.332444,0.001986,0.323033,0.001078,0.330716,0.001835,0.002431,0.000288,0.001262,0.000229,...,0.000000,0.881620,0.142986,0.0,0.000000,False,0.0,10,0.966788,0.97940


In [16]:
# TODO: ask for a code review
# std = np.sqrt(np.mean(x**2) - np.mean(x)**2)

def calc_overall_stdev(row, suffix=''):
    return np.sqrt(                 # 5. apply root square to get the SD from the variance
        (np.mean(                   # 3. divide by the number of trained model
            (row[f'stdev{suffix}']**2)       # 1. cancel the overall square root
             + row[f'mean{suffix}']**2))     # 2. add the mean and get the sum or squares
        - np.mean(row[f'mean{suffix}'])**2)  # 4. substrct the square of the overall mean

In [17]:
SHUFFLE_METHOD_SUFFIX = ['_layer', '_layer_nonzero']
for suffix in SHUFFLE_METHOD_SUFFIX:
    statistics_df[f'overall_std{suffix}'] = df_grpby_model.apply(calc_overall_stdev, suffix)

for suffix in SHUFFLE_METHOD_SUFFIX:
    statistics_df[f'is_stable{suffix}'] = (statistics_df[(f'is_sig{suffix}', 'mean')] == 1)

statistics_df = statistics_df.loc[TAGS_COMMANDS.keys(), :]

In [18]:
statistics_df

,"(ncut, mean)","(ncut, std)","(mean_layer, mean)","(mean_layer, std)","(mean_layer_nonzero, mean)","(mean_layer_nonzero, std)","(stdev_layer, mean)","(stdev_layer, std)","(stdev_layer_nonzero, mean)","(stdev_layer_nonzero, std)",...,"(is_sig_layer, std)","(is_sig_layer_nonzero, mean)","(is_sig_layer_nonzero, std)",n_models,train_acc,test_acc,overall_std_layer,overall_std_layer_nonzero,is_stable_layer,is_stable_layer_nonzero
model,,,,,,,,,,,,,,,,,,,,,
MNIST,0.333068,0.002811,0.330491,0.001174,0.331605,0.002762,0.002266,0.000253,0.001045,0.000201,...,0.000000,False,0.0,10,1.000000,0.98357,0.002536,0.002827,False,False
CIFAR10,0.306362,0.009507,0.312830,0.006120,0.296695,0.013320,0.001239,0.000184,0.000581,0.000188,...,0.316228,False,0.0,10,0.650088,0.41523,0.005939,0.012651,False,False
FASHION,0.319421,0.003331,0.312592,0.001731,0.317683,0.002728,0.002186,0.000274,0.000926,0.000157,...,0.000000,False,0.0,10,0.982630,0.89344,0.002746,0.002753,False,False
MNIST+DROPOUT,0.332444,0.001986,0.323033,0.001078,0.330716,0.001835,0.002431,0.000288,0.001262,0.000229,...,0.000000,False,0.0,10,0.966788,0.97940,0.002651,0.002161,False,False
CIFAR10+DROPOUT,0.289859,0.014990,0.308491,0.008694,0.257023,0.024589,0.000955,0.000167,0.001108,0.000193,...,0.316228,False,0.0,10,0.427050,0.42189,0.008304,0.023354,False,False
FASHION+DROPOUT,0.311667,0.003239,0.312478,0.001810,0.311663,0.002760,0.001896,0.000245,0.001072,0.000263,...,0.000000,False,0.0,10,0.863268,0.86921,0.002568,0.002840,False,False


In [19]:
df.to_csv(f'../results/mlp-clustering-stability-two-shuffling-methods-all-samples-K-{N_CLUSTERS}.csv')
statistics_df.to_csv(f'../results/mlp-clustering-stability-two-shuffling-methods-statistics-K-{N_CLUSTERS}.csv')

### Model Stability Statistics

In [20]:
statistics_df

,"(ncut, mean)","(ncut, std)","(mean_layer, mean)","(mean_layer, std)","(mean_layer_nonzero, mean)","(mean_layer_nonzero, std)","(stdev_layer, mean)","(stdev_layer, std)","(stdev_layer_nonzero, mean)","(stdev_layer_nonzero, std)",...,"(is_sig_layer, std)","(is_sig_layer_nonzero, mean)","(is_sig_layer_nonzero, std)",n_models,train_acc,test_acc,overall_std_layer,overall_std_layer_nonzero,is_stable_layer,is_stable_layer_nonzero
model,,,,,,,,,,,,,,,,,,,,,
MNIST,0.333068,0.002811,0.330491,0.001174,0.331605,0.002762,0.002266,0.000253,0.001045,0.000201,...,0.000000,False,0.0,10,1.000000,0.98357,0.002536,0.002827,False,False
CIFAR10,0.306362,0.009507,0.312830,0.006120,0.296695,0.013320,0.001239,0.000184,0.000581,0.000188,...,0.316228,False,0.0,10,0.650088,0.41523,0.005939,0.012651,False,False
FASHION,0.319421,0.003331,0.312592,0.001731,0.317683,0.002728,0.002186,0.000274,0.000926,0.000157,...,0.000000,False,0.0,10,0.982630,0.89344,0.002746,0.002753,False,False
MNIST+DROPOUT,0.332444,0.001986,0.323033,0.001078,0.330716,0.001835,0.002431,0.000288,0.001262,0.000229,...,0.000000,False,0.0,10,0.966788,0.97940,0.002651,0.002161,False,False
CIFAR10+DROPOUT,0.289859,0.014990,0.308491,0.008694,0.257023,0.024589,0.000955,0.000167,0.001108,0.000193,...,0.316228,False,0.0,10,0.427050,0.42189,0.008304,0.023354,False,False
FASHION+DROPOUT,0.311667,0.003239,0.312478,0.001810,0.311663,0.002760,0.001896,0.000245,0.001072,0.000263,...,0.000000,False,0.0,10,0.863268,0.86921,0.002568,0.002840,False,False


### Appendix - All Samples

In [21]:
df

,model,train_acc,train_loss,test_acc,test_loss,ncut,ave_in_out,n_samples,mean_layer,percentile_layer,stdev_layer,z_score_layer,is_sig_layer,mean_layer_nonzero,percentile_layer_nonzero,stdev_layer_nonzero,z_score_layer_nonzero,is_sig_layer_nonzero,cohen_d
0,MNIST,1.000000,0.000048,0.9835,0.095332,0.329965,2.530629,320,0.329872,0.563863,0.002359,0.039226,False,0.329494,0.595016,0.001083,0.434404,False,0.483902
1,MNIST,1.000000,0.000056,0.9847,0.090697,0.330161,2.528825,320,0.329599,0.563863,0.002051,0.273888,False,0.328940,0.875389,0.000903,1.351846,False,0.518073
2,MNIST,1.000000,0.000059,0.9830,0.099833,0.333930,2.494644,320,0.328610,0.968847,0.002429,2.190334,False,0.332132,0.968847,0.001124,1.599889,False,0.474365
3,MNIST,1.000000,0.000062,0.9828,0.099546,0.331197,2.519351,320,0.329903,0.626168,0.002327,0.556271,False,0.329517,1.000000,0.000927,1.812551,False,0.513064
4,MNIST,1.000000,0.000081,0.9845,0.097756,0.337609,2.462010,320,0.332344,1.000000,0.002243,2.347679,False,0.335694,0.875389,0.001318,1.452518,False,0.450070
5,MNIST,1.000000,0.000048,0.9835,0.095626,0.331404,2.517462,320,0.331395,0.408100,0.002754,0.003563,False,0.328806,1.000000,0.000848,3.063931,False,0.531824
6,MNIST,1.000000,0.000056,0.9838,0.087238,0.335151,2.483726,320,0.330888,0.968847,0.002116,2.014740,False,0.333080,1.000000,0.001018,2.033987,False,0.495887
7,MNIST,1.000000,0.000061,0.9838,0.100444,0.332732,2.505426,320,0.332105,0.626168,0.001924,0.325583,False,0.333334,0.221184,0.000755,-0.797120,False,0.554175
8,MNIST,1.000000,0.000047,0.9828,0.093142,0.331260,2.518777,320,0.330194,0.657321,0.001992,0.535228,False,0.329284,0.937695,0.001082,1.826329,False,0.485221
9,MNIST,1.000000,0.000049,0.9833,0.102283,0.337271,2.464977,320,0.330002,0.968847,0.002464,2.949653,False,0.335770,0.844237,0.001393,1.077225,False,0.436860


### Unpruned results without shuffling
#### TODO: Refactor
- [ ] Combine with the unpruned table

In [22]:
if False:
    unpruned_results = {}

    for (model_tag, _), dataset_tag in zip(TAGS_COMMANDS.items(), tqdm(DATASETS_TAGS)):
        model_paths = get_model_path(model_tag, filter_='all', model_base_path='/scratch/shlomi/models/')
        unpruned_results[model_tag] = [run_spectral_cluster(path / f'{dataset_tag}-mlp-unpruned-weights.pckl',
                                                   with_shuffle=False)
                for path in tqdm(model_paths[-N_TRAINED_MODELS:])]

    unpruned_model_dfs = []

    for model_tag, model_results in unpruned_results.items():
        _, metrics  = zip(*model_results)
        unpruned_model_dfs.append(pd.DataFrame(metrics).assign(model=model_tag))

    unpruned_df = pd.concat(unpruned_model_dfs, ignore_index=True)

    unpruned_df_grpby_model = unpruned_df.groupby('model')


    unpruned_statistics_df = pd.concat([(unpruned_df_grpby_model[['ncut']]
                                 .agg(['mean', 'std'])),
                                unpruned_df_grpby_model.size().rename('n_models'),
                                (unpruned_df_grpby_model[['train_acc', 'test_acc']]
                                 .agg('mean'))],
                               axis=1)

    unpruned_statistics_df = unpruned_statistics_df.loc[TAGS_COMMANDS.keys(), :]

    unpruned_statistics_df

In [23]:
# unpruned_df.to_csv(f'../results/mlp-clustering-stability-all-samples-unpruned-K-{N_CLUSTERS}.csv')
# unpruned_statistics_df.to_csv(f'../results/mlp-clustering-stability-statistics-unpruned-K-{N_CLUSTERS}.csv')